<a href="https://colab.research.google.com/github/yoly2020/ocr_test_project/blob/main/OCR2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###预处理

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


加载包

In [ ]:
# 1
import numpy as np
from PIL import Image
from collections import defaultdict
from itertools import product
from sklearn.model_selection import train_test_split
import shutil
import re
import glob
import os

In [ ]:
import os
os.chdir('/content/gdrive/My Drive/OCR Project/Logo Project/data/')

In [ ]:
# 2
from scipy import ndimage
from six.moves import cPickle as pickle
import re
import common 

In [ ]:
# 3
import tensorflow as tf
import numpy as np
from six.moves import cPickle as pickle
from six.moves import range
import sys
import os

 常数定义

In [ ]:
# CNN input size constants
CNN_IN_WIDTH=64
CNN_IN_HEIGHT=32
CNN_IN_CH=3

# train valid test size constants
TRAIN_SIZE=5000 # total train file: 163169
VALID_SIZE=500
TEST_SIZE=500 # total test file: 54425

# data augmentation constants
DATA_AUG_POS_SHIFT_MIN=-2
DATA_AUG_POS_SHIFT_MAX=2
DATA_AUG_SCALES=[0.9,1.1]
DATA_AUG_ROT_MIN=-15
DATA_AUG_ROT_MAX=15

# 文件位置相关常量
TRAIN_DIR='flickr_logos_27_dataset'
TRAIN_IMAGE_DIR=os.path.join(TRAIN_DIR,'flickr_logos_27_dataset_images')
CROPPED_AUG_IMAGE_DIR=os.path.join(TRAIN_DIR,'flickr_logos_27_dataset_cropped_augmented_images')
ANNOT_FILE='flickr_logos_27_dataset_training_set_annotation.txt'
NONE_IMAGE_DIR=os.path.join(TRAIN_DIR,'empty_dir')
NUM_OF_NONE_IMAGES=1000

# class constants
CLASS_NAME = ['Adidas', 'Apple', 'BMW', 'Citroen', 'Cocacola', 'DHL', 'Fedex', 'Ferrari',
    'Ford', 'Google', 'HP', 'Heineken', 'Intel', 'McDonalds', 'Mini', 'Nbc',
    'Nike', 'Pepsi', 'Porsche', 'Puma', 'RedBull', 'Sprite', 'Starbucks',
    'Texaco', 'Unicef', 'Vodafone', 'Yahoo']

# image相关常量
PIXEL_DEPTH=255.0

# 用Pickle保存训练用的数据相关常量
PICKLE_FILENAME='deep_logo.pickle'

# tensorflow相关常量
TF_TRAIN_DIR='flickr_logos_27_dataset'
TF_MAX_STEPS=5001
TF_IMAGE_WIDTH=64
TF_IMAGE_HEIGHT=32
TF_NUM_CLASSES=27
TF_IMAGE_CHANNELS=3
TF_LEARNING_RATE=0.0001
TF_BATCH_SIZE=64
TF_PATCH_SIZE=5 # 类似window size，每次cnnfilter作用于图片的一个patch大小的地方，而不是整个图片，对于kernel来说patch就是input

###第一步，image augmentation and cropping

定义所需函数

In [ ]:
# 解析读入的有image标记的txt文件'flickr_logos_27_dataset_training_set_annotation.txt'

def parse_annot(annot):
  file_name=annot[0].decode('utf-8') # 解析utf8格式的string
  class_name=annot[1].decode('utf-8')
  train_subset_class=annot[2].decode('utf-8')
  return file_name,class_name,train_subset_class


In [ ]:
# 从标记文件中获取rectangle的信息

def get_rect(annot):
  rect=defaultdict(int)
  x1,y1,x2,y2=rect_coord(annot[3:]) # top-left -> bottom-right
  cx,cy,width,height=center_width_height(x1,y1,x2,y2)
  rect['x1']=x1
  rect['y1']=y1
  rect['x2']=x2
  rect['y2']=y2
  rect['cx']=cx
  rect['cy']=cy
  rect['width']=width
  rect['height']=height
  
  return rect

In [ ]:
# 计算rectangle的中心坐标和高度、宽度

def center_width_height(x1,y1,x2,y2):
  cx=x1+(x2-x1)//2 # 向下取整的整除
  cy=y1+(y2-y1)//2
  width=(x2-x1)
  height=(y2-y1)
  return cx,cy,width,height

In [ ]:
# 用pillow包来crop（取一部分）图像
# 切换标记矩形的中心位置并获取变换过后的标记矩形中的这部分图像
# resize以适应cnn输入大小

def aug_pos(annot,img):
  aug_pos_imgs=[] # 存储cropped images
  aug_pos_suffixes=[] # 用于查看每次的shift值
  rect=get_rect(annot)

  for sx,sy in product(range(DATA_AUG_POS_SHIFT_MIN,DATA_AUG_POS_SHIFT_MAX),
                       range(DATA_AUG_POS_SHIFT_MIN,DATA_AUG_POS_SHIFT_MAX)):
    new_cx=rect['cx']+sx
    new_cy=rect['cy']+sy

    cropped_img=img.crop((new_cx-rect['width']//2,new_cy-rect['height']//2,
                          new_cx+rect['width']//2,new_cy+rect['height']//2))
    resized_img=cropped_img.resize((CNN_IN_WIDTH,CNN_IN_HEIGHT))

    aug_pos_imgs.append(resized_img)
    aug_pos_suffixes.append('p'+str(sx)+str(sy))
    cropped_img.close()

  return aug_pos_imgs,aug_pos_suffixes


In [ ]:
# 用pillow包来crop（取一部分）图像
# scale标记矩形并获取变换过后的标记矩形中这部分图像
# resize以适应cnn输入大小

def aug_scale(annot,img):
  aug_scale_imgs=[]
  aug_scale_suffixes=[]
  rect=get_rect(annot)
  
  for s in DATA_AUG_SCALES:
    new_width=int(rect['width']*s)
    new_height=int(rect['height']*s)

    cropped_img=img.crop((rect['cx']-new_width//2,rect['cy']-new_height//2,
                          rect['cx']+new_width//2,rect['cy']-new_height//2))
    resized_img=cropped_img.resize((CNN_IN_WIDTH,CNN_IN_HEIGHT))

    aug_scale_imgs.append(resized_img)
    aug_scale_suffixes.append('s'+str(s))
    cropped_img.close()

  return aug_scale_imgs,aug_scale_suffixes


In [ ]:
# 用pillow包来crop（取一部分）图像
# 旋转图片并获取标记矩形中的这部分图像
# resize以适应cnn输入大小

def aug_rot(annot,img):
  aug_rot_imgs=[]
  aug_rot_suffixes=[]
  rect=get_rect(annot)

  for r in range(DATA_AUG_ROT_MIN,DATA_AUG_ROT_MAX):
    rotated_img=img.rotate(r)
    cropped_img=rotated_img.crop((rect['cx']-rect['width']//2,rect['cy']-rect['height']//2,
                                  rect['cx']+rect['width']//2,rect['cy']+rect['height']//2))
    resized_img=cropped_img.resize((CNN_IN_WIDTH,CNN_IN_HEIGHT))
    
    aug_rot_imgs.append(resized_img)
    aug_rot_suffixes.append('r'+str(r))
    rotated_img.close()
    cropped_img.close()

  return aug_rot_imgs,aug_rot_suffixes

In [ ]:
# crop图片以获取原图中标记矩形位置部分的图像
# resize以适应cnn输入大小

def crop_logos(annot,img):
  x1,y1,x2,y2=rect_coord(annot[3:])

  cropped_img=img.crop((x1,y1,x2,y2))
  resized_img=cropped_img.resize((CNN_IN_WIDTH,CNN_IN_HEIGHT))
  cropped_suffix='p00'
  
  return [resized_img],[cropped_suffix]

In [ ]:
def rect_coord(annot_part):
  # annot_part表示annot文件中属于标记矩形的坐标的那部分信息
  return list(map(int,annot_part)) # 是用map函数分别把annot文件中的坐标信息变成整数，再将结果放入list中

In [ ]:
# 判断图片是否需要skip掉，判断标准为标记矩形的长宽是否不大于0

def is_skip(annot_part):
  x1,y1,x2,y2=rect_coord(annot_part)
  _,_,width,height=center_width_height(x1,y1,x2,y2)

  if width<=0 or height <=0:
    return True
  else:
    return False

In [ ]:
# 处理过后的图片文件的保存命名格式

def save_img(annot,cnt,*args):
  # *args表示variable个数的argument
  file_name,class_name,train_subset_class=parse_annot(annot)
  dest_dir=os.path.join(CROPPED_AUG_IMAGE_DIR,class_name) # destination directory保存处理过的图片的目标文件夹
  if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)
  
  for i,arg in enumerate(args):
    for img,suffix in zip(arg[0],arg[1]):
      save_file_name='_'.join([file_name.split('.')[0],class_name,train_subset_class,str(cnt),suffix]) + os.path.splitext(file_name)[1]
      img.save(os.path.join(dest_dir,save_file_name)) # os.path.splitext()[1]获得root name i.e. '.jpg'


In [ ]:
# 是用pillow包中的close函数来关闭图片的批量处理函数

def close_img(*args):
  for imgs in args:
    for img in imgs:
      img.close()

In [ ]:
def crop_and_aug(annot_train):
  cnt_per_file=defaultdict(int)

  for annot in annot_train:
    file_name,_,_=parse_annot(annot)
    cnt_per_file[file_name]+=1 # 相同图片计数，因为同一张图片上可能有多个logo

    if is_skip(annot[3:]):
      print('Skip', file_name)
      continue
    
    img=Image.open(os.path.join(TRAIN_IMAGE_DIR,file_name))

    cropped_imgs,cropped_suffixes=crop_logos(annot,img)

    shifted_imgs,shifted_suffixes=aug_pos(annot,img)

    scaled_imgs,scaled_suffixes=aug_scale(annot,img)

    rotated_imgs,rotated_suffixes=aug_rot(annot,img)

    save_img(annot,cnt_per_file[file_name],[cropped_imgs,cropped_suffixes],[shifted_imgs,shifted_suffixes],
             [scaled_imgs,scaled_suffixes],[rotated_imgs,rotated_suffixes])
    
    close_img([img],cropped_imgs,shifted_imgs,scaled_imgs,rotated_imgs)

In [ ]:
def crop_none():
    none_img_classes = [
        cn.decode('utf-8')
        for cn in np.loadtxt(
            os.path.join(NONE_IMAGE_DIR, 'ClassName.txt'), dtype='a')
    ]

    dst_dir = os.path.join(CROPPED_AUG_IMAGE_DIR, 'None')
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)

    for none_class in none_img_classes:
        none_dir = os.path.join(NONE_IMAGE_DIR, none_class[1:])
        none_imgs = [
            os.path.join(none_dir, img) for img in os.listdir(none_dir)
            if re.search('\.jpg', img)
        ]
        none_imgs = np.random.choice(none_imgs, 10)
        for none_img in none_imgs:
            im = Image.open(none_img)
            if im.mode != "RGB":
                im = im.convert("RGB")
            w, h = im.size
            cw, ch = w // 2, h // 2
            cropped_im = im.crop(
                (cw - CNN_IN_WIDTH // 2, ch - CNN_IN_HEIGHT // 2,
                 cw + CNN_IN_WIDTH // 2, ch + CNN_IN_HEIGHT // 2))
            dst_fn = os.path.basename(none_img)
            cropped_im.save(os.path.join(dst_dir, dst_fn))


def crop_and_aug_with_none(annot_train, with_none=False):
    # root directory to save processed images
    if not os.path.exists(CROPPED_AUG_IMAGE_DIR):
        os.makedirs(CROPPED_AUG_IMAGE_DIR)

    # crop images and apply augmentation
    crop_and_aug(annot_train)

    # crop images of none class
    if with_none:
        crop_none()

    # print results
    org_imgs = [img for img in os.listdir(TRAIN_IMAGE_DIR)]
    crop_and_aug_imgs = [
        fname
        for root, dirs, files in os.walk(CROPPED_AUG_IMAGE_DIR)
        for fname in glob.glob(os.path.join(root, '*.jpg'))
    ]
    print('original: %d' % (len(org_imgs)))
    print('cropped: %d' % (len(crop_and_aug_imgs)))


In [ ]:
# 将aug过的图片文件夹中每个class文件夹中的img分成train和test部分并放入名为train和test的文件夹
# 文件夹层级类似于：
# --flickr_logos_27_dataset_cropped_augmented_images
# ---Adidas
# ----test
# ----train

def do_train_test_split():
  class_names=[cls for cls in os.listdir(CROPPED_AUG_IMAGE_DIR)]

  for class_name in class_names:
    if os.path.exists(os.path.join(CROPPED_AUG_IMAGE_DIR,class_name,'train')):
      continue
    if os.path.exists(os.path.join(CROPPED_AUG_IMAGE_DIR,class_name,'test')):
      continue
    
    imgs=[img for img in os.listdir(os.path.join(CROPPED_AUG_IMAGE_DIR,class_name))]

    # train=0.75 test=0.25

    train_imgs,test_imgs=train_test_split(imgs)

    os.makedirs(os.path.join(CROPPED_AUG_IMAGE_DIR,class_name,'train'))
    os.makedirs(os.path.join(CROPPED_AUG_IMAGE_DIR,class_name,'test'))

    for img in train_imgs:
      dest=os.path.join(CROPPED_AUG_IMAGE_DIR,class_name,'train')
      src=os.path.join(CROPPED_AUG_IMAGE_DIR,class_name,img)

      shutil.move(src,dest)

    for img in test_imgs:
      dest=os.path.join(CROPPED_AUG_IMAGE_DIR,class_name,'test')
      src=os.path.join(CROPPED_AUG_IMAGE_DIR,class_name,img)

      shutil.move(src,dest)

执行

In [ ]:
annot_train=np.loadtxt(os.path.join(TRAIN_DIR,ANNOT_FILE),dtype='a')
print('train_annotation: %d, %d' % (annot_train.shape))

train_annotation: 4536, 7


In [ ]:
crop_and_aug_with_none(annot_train)

Skip 2662264721.jpg
Skip 2662264721.jpg
Skip 2662264721.jpg
Skip 2662264721.jpg
Skip 2662264721.jpg
original: 1079
cropped: 217488


In [ ]:
do_train_test_split()

###第二步，生成用于train、valid、test的数据

定义所需函数

In [ ]:
# 从每个train或者test的文件夹中load logo图片并将处理过后的图片其放入dataset

def load_logo(data_dir):
  img_files=os.listdir(data_dir)
  dataset=np.ndarray(shape=(len(img_files),CNN_IN_HEIGHT,CNN_IN_WIDTH,CNN_IN_CH),dtype=np.float32)
  
  print(data_dir)
  
  num_imgs=0

  for img in img_files:
    img_file=os.path.join(data_dir,img)
    try:
      img_data=(ndimage.imread(img_file).astype(float)-PIXEL_DEPTH/2)/PIXEL_DEPTH
      if img_data.shape!=(CNN_IN_HEIGHT,CNN_IN_WIDTH,CNN_IN_CH):
        raise Exception('Unexpected image shape: %s' % str(img_data.shape))
      dataset[num_imgs,:,:]=img_data
      num_imgs+=1
    except IOError as e:
      print('Could not read:', img_file,':',e,'-it\'s ok, skipping.')
  
  dataset=dataset[0:num_imgs,:,:]
  print('Full dataset tensor: ',dataset.shape)
  print('Mean: ',np.mean(dataset))
  print('Standard deviation: ',np.std(dataset))
  # print('测试img_data到底是个啥：img_data=',img_data.shape,'\n',img_data)

  return dataset

In [ ]:
# 把每个train、test文件夹中的数据以pickle格式保存在与train、test文件夹同层级的文件夹中
# 返回存储着每个保存的pickle文件的全名的数组

def maybe_pickle(data_dirs,force=False):
  dataset_names=[]

  for dir in data_dirs:
    set_filename=dir+'.pickle'
    dataset_names.append(set_filename)
    
    if os.path.exists(set_filename) and not force:
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)

      dataset=load_logo(dir)

      try:
        with open(set_filename,'wb') as pickle_file:
          pickle.dump(dataset,pickle_file,pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to',set_filename,':',e)
  
  return dataset_names


In [ ]:
# 建立包含num_rows个（grayscale）图片的数组和其对应标签数组

def make_arrays(num_rows,img_width,img_height,img_ch=1):
  if num_rows:
    dataset=np.ndarray((num_rows,img_height,img_width,img_ch),dtype=np.float32)
    labels=np.ndarray(num_rows,dtype=np.int32)
  else:
    dataset,labels=None,None
  return dataset,labels

In [ ]:
# 建立包含train（，valid）和test的数据的数组：从许多个class文件夹中的pickle文件读取，并分配以及分别合并在一起

def merge_datasets(pickle_files,train_size,valid_size=0):
  num_classes=len(pickle_files)

  valid_dataset,valid_labels=make_arrays(valid_size,CNN_IN_WIDTH,CNN_IN_HEIGHT,CNN_IN_CH)
  train_dataset,train_labels=make_arrays(train_size,CNN_IN_WIDTH,CNN_IN_HEIGHT,CNN_IN_CH)
  
  valid_size_per_class=valid_size//num_classes
  train_size_per_class=train_size//num_classes

  start_valid,start_train=0,0
  end_valid,end_train=valid_size_per_class,train_size_per_class
  end_length=valid_size_per_class+train_size_per_class

  for label,pickle_file in enumerate(pickle_files):
    try:
      with open(pickle_file,'rb') as file:
        logo_set=pickle.load(file)
        np.random.shuffle(logo_set) # 随机打乱每个class下的logo图片
        if valid_dataset is not None:
          # valid只有在传入train的时候从train里分配，传入test的时候不分配（虽然传入test的时候里面的相关变量还是叫train）
          valid_logos=logo_set[:valid_size_per_class,:,:,:]

          valid_dataset[start_valid:end_valid,:,:,:]=valid_logos
          
          valid_labels[start_valid:end_valid]=label # 都是同一个class啦

          start_valid+=valid_size_per_class # 移动被填写数组的光标
          end_valid+=valid_size_per_class
        
        train_logo=logo_set[valid_size_per_class:end_length,:,:,:] # train从logoset中valid的后面取

        train_dataset[start_train:end_train,:,:,:]=train_logo

        train_labels[start_train:end_train]=label

        start_train+=train_size_per_class
        end_train+=train_size_per_class

    except Exception as e:
      print('Unable to process data from', pickle_file,':',e)
      raise
  return valid_dataset,valid_labels,train_dataset,train_labels







In [ ]:
# 打乱dataset和label数组，因为之前在merge_datasets里虽然用shuffle打乱了每个class底下的图片，但是还没有把全部class打乱(之前的洗牌在每个class里进行，现在的洗牌对所有牌进行)

def randomize(dataset,labels):
  permutation=np.random.permutation(labels.shape[0])
  
  shuffled_dataset=dataset[permutation,:,:]
  shuffled_labels=labels[permutation]

  return shuffled_dataset,shuffled_labels


In [ ]:
# 将融合后的train，valid，test的相关数据用pickle保存了

def save_pickle(train_dataset,train_labels,valid_dataset,valid_labels,test_dataset,test_labels):
  try:
    file=open(PICKLE_FILENAME,'wb')
    
    save={
        'train_dataset':train_dataset,
        'train_labels':train_labels,
        'valid_dataset':valid_dataset,
        'valid_labels':valid_labels,
        'test_dataset':test_dataset,
        'test_labels':test_labels,
    }

    pickle.dump(save,file,pickle.HIGHEST_PROTOCOL)

    file.close()

  except Exception as e:
    print('Unable to save data to',PICKLE_FILENAME,':',e)
    raise


执行

In [ ]:
train_dirs=[os.path.join(CROPPED_AUG_IMAGE_DIR,class_name,'train') for class_name in CLASS_NAME]
test_dirs=[os.path.join(CROPPED_AUG_IMAGE_DIR,class_name,'test') for class_name in CLASS_NAME]


In [ ]:
train_datasets=maybe_pickle(train_dirs)
test_datasets=maybe_pickle(test_dirs)

In [ ]:
valid_dataset,valid_labels,train_dataset,train_labels=merge_datasets(train_datasets,TRAIN_SIZE,VALID_SIZE)

In [ ]:
_,_,test_dataset,test_labels=merge_datasets(test_datasets,TEST_SIZE)

In [ ]:
train_dataset,train_labels=randomize(train_dataset,train_labels)
valid_dataset,valid_labels=randomize(valid_dataset,valid_labels)
test_dataset,test_labels=randomize(test_dataset,test_labels)

In [ ]:
save_pickle(train_dataset,train_labels,valid_dataset,valid_labels,test_dataset,test_labels)
statinfo=os.stat(PICKLE_FILENAME)
print('Compressed pickle file size:',statinfo.st_size)

###第三步，模型训练

定义所需函数

In [ ]:
# 计算预测的accuracy

def accuracy(predictions,labels):
  return (100*np.sum(np.argmax(predictions,1)==np.argmax(labels,1))/predictions.shape[0]) # 每行找出最大的下标进行比较,axis=1

In [ ]:
# 重编dataset和labels的形状

def reformat(dataset,labels):
  dataset=dataset.reshape((-1,TF_IMAGE_WIDTH,TF_IMAGE_HEIGHT,TF_IMAGE_CHANNELS)).astype(np.float32) # -1表示这个dimension的大小由numpy自己计算决定
  labels=(np.arange(TF_NUM_CLASSES)==labels[:,None]).astype(np.float32) # 以1.0或0.0表示labels里的数是否在[0，27)内,形成一个有原labels.shape[0]行、27列，每行只有一个1.0的（二维）数组（类似于one-hot coding）

  return dataset,labels

In [ ]:
# 建立模型

def my_model(data,filter_conv1,bias_conv1,filter_conv2,bias_conv2,filter_conv3,bias_conv3,filter_fc1,bias_fc1,filter_fc2,bias_fc2):
  # Layer 1
  h_conv1=tf.nn.relu(tf.nn.conv2d(data,filter_conv1,[1,1,1,1],padding='SAME')+bias_conv1)
  print('filter_conv1:',filter_conv1.shape)
  print('bias_conv1:',bias_conv1.shape)
  print('bias.conv1:',bias_conv1)
  print('h_conv1:',h_conv1.get_shape().as_list())

  h_pool1=tf.nn.max_pool(h_conv1,ksize=[1,2,2,1],strides=[1,1,2,1],padding='SAME')

  # Layer 2
  h_conv2=tf.nn.relu(tf.nn.conv2d(h_pool1,filter_conv2,[1,1,1,1],padding='SAME')+bias_conv2)
  h_pool2=tf.nn.max_pool(h_conv2,ksize=[1,1,2,1],strides=[1,1,2,1],padding='SAME')

  # Layer 3
  h_conv3=tf.nn.relu(tf.nn.conv2d(h_pool2,filter_conv3,[1,1,1,1],padding='SAME')+bias_conv3)
  h_pool3=tf.nn.max_pool(h_conv3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

  # Fully Connected Layer
  conv_layer_flat=tf.reshape(h_pool3,[-1,16*4*128])
  h_fc1=tf.nn.relu(tf.matmul(conv_layer_flat,filter_fc1)+bias_fc1)

  # Output Layer
  out=tf.matmul(h_fc1,filter_fc2)+bias_fc2

  return out

In [ ]:
# 从之前保存数据的deep_logo.pickle文件中读取数据

def read_data():
  with open(PICKLE_FILENAME,'rb') as file:
    save=pickle.load()

    train_dataset=save['train_dataset']
    train_labels=save['train_labels']
    valid_dataset=save['valid_dataset']
    valid_labels=save['valid_labels']
    test_dataset=save['test_dataset']
    test_labels=save['test_labels']

    del save

    print('Training set',train_dataset.shape,train_labels.shape)
    print('Valid set',valid_dataset.shape,valid_labels.shape)
    print('Test set',test_dataset.shape,test_labels.shape)

  return [train_dataset,valid_dataset,test_dataset],[train_labels.valid_labels,test_labels]

执行

In [ ]:
if len(sys.argv) > 1:
  f=np.load(sys.argv[1])

  initial_weights=[
                   f[n] for n in sorted(f.files,key=lambda s: int(s[4:]))
  ]
else:
  initial_weights=None
  

In [ ]:
dataset,labels=read_data()

In [ ]:
train_dataset,train_labels=reformat(dataset[0],labels[0])
valid_dataset,valid_labels=reformat(dataset[1],labels[1])
test_dataset,test_labels=reformat(dataset[2],labels[2])

print('Training set',train_dataset.shape,train_labels.shape)
print('Valid set',valid_dataset.shape,valid_labels.shape)
print('Test set',test_dataset.shape,test_labels.shape)

In [ ]:
# 关于training model的一些处理

graph=tf.Graph()

with graph.as_default():
  # 1. Variables设定

  # truncated_normal: 以mean为base做正态分布然后随机取值，大于2个标准差standard deviation（95%）的数据被抛弃（所以叫truncated）并重新选值
  filter_conv1=tf.Variable(tf.truncated_normal([TF_PATCH_SIZE,TF_PATCH_SIZE,TF_IMAGE_CHANNLES,48],stddev=0.1)) # 那个1D array是the shape of the output tensor
  bias_conv1=tf.Variable(tf.constant(0.1,shape=[48]))

  filter_conv2=tf.Variable(tf.truncated_normal([TF_PATCH_SIZE,TF_PATCH_SIZE,48,64],stddev=0.1))
  bias_conv2=tf.Variable(tf.constant(0.1,shape=[64]))

  filter_conv3=tf.Variable(tf.truncated_normal([TF_PATCH_SIZE,TF_PATCH_SIZE,64,128],stddev=0.1))
  bias_conv3=tf.Variable(tf.constant(0.1,shape=[2048]))

  filter_fc1=tf.Variable(tf.truncated_normal([16*4*128,2048],stddev=0.1))
  bias_fc1=tf.Variable(tf.constant(0.1,shape=[2048]))

  filter_fc2=tf.Variable(tf.truncated_normal([2048,TF_NUM_CLASSES]))
  bias_fc2=tf.Variable(tf.constant(0.1,shape=[TF_NUM_CLASSES]))

  # 2. Parameters设定

  params=[filter_conv1,bias_conv1,filter_conv2,bias_conv2,filter_conv3,bias_conv3,filter_fc1,bias_fc1,filter_fc2,bias_fc2]

  # 3. Initial Weights设定

  if initial_weights is not None:
    assert len(params) == len(initial_weights)
    assign_ops=[w.assign(v) for w,v in zip(params,initial_weights)] # assign_operations creates a tf.Operation that you have to explicitly run to update the variable.

  # 4. Input Data设定
  
  tf_train_dataset=tf.placeholder(tf.float32,shape=(TF_BATCH_SIZE,TF_IMAGE_WIDTH,TF_IMAGE_HEIGHT,TF_IMAGE_CHANNELS))
  tf_train_labels=tf.placeholder(tf.float32,shape=(TF_BATCH_SIZE,TF_NUM_CLASSES))

  tf_valid_dataset=tf.constant(valid_dataset) # constant tensor
  
  tf_test_dataset=tf.constant(test_dataset)

  # 5. Training时相关计算量的设定
  
  logits=my_model(tf_train_dataset,filter_conv1,bias_conv1,filter_conv2,bias_conv2,filter_conv3,bias_conv3,filter_fc1,bias_fc1,filter_fc2,bias_fc2)

  with tf.name_scope('loss'):
    loss=tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=tf_train_labels))

    tf.summary.scalar('loss',loss) # 将loss的值以标量的形式保存在tensorboard（tensorflow的可视化工具）里
  
  optimizer=tf.train.AdamOptimizer(TF_LEARNING_RATE).minimize(loss)

  # 6. 对training、validation、testing data的predictions设定（注意logits是没有经过normalization的log probabilities
  
  train_prediction=tf.nn.softmax(logits)
  valid_prediction=tf.nn.softmax(my_model(tf_valid_dataset,filter_conv1,bias_conv1,filter_conv2,bias_conv2,filter_conv3,bias_conv3,filter_fc1,bias_fc1,filter_fc2,bias_fc2))
  test_prediction=tf.nn.softmax(my_model(tf_test_dataset,filter_conv1,bias_conv1,filter_conv2,bias_conv2,filter_conv3,bias_conv3,filter_fc1,bias_fc1,filter_fc2,bias_fc2))

  # 7. 将所有summary信息merge并保存起来

  merged=tf.summary.merge_all()
  # merge_all 可以将所有summary全部保存到磁盘，以便tensorboard显示

  train_writer=tf.summary.FileWriter(TF_TRAIN_DIR+'/train') # 指定一个文件加用来保存图

  # 8. Add ops to save and restore all the variables.
  
  saver=tf.train.Saver()

In [ ]:
# Do Training

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run() # 加了这步之后所有variables才真正hold the values you declare to them（相当于之前只是声明了但没有set）

  if initial_weights is not None:
    session.run(assign_ops)
    print('Initialize by pre-learned values')

  else:
    print('Initialized')
  
  for step in range(TF_MAX_STEPS):
    offset=(step*TF_BATCH_SIZE) % (train_labels.shape[0]-TF_BATCH_SIZE)

    if step <=5:
      print('offset:',offset)
      print('train_labels.shape[0]',train_labels.shape[0])
    
    batch_data=train_dataset[offset:(offset+TF_BATCH_SIZE),:,:,:]

    batch_labels=train_labels[offset:(offset+TF_BATCH_SIZE),:]

    feed_dict={
        tf_train_dataset:batch_data,
        tf_train_labels:batch_labels
    }

    try:
      _,l,predictions=session.run([optimizer,loss,train_prediction],feed_dict=feed_dict) # l: loss

      if step%50 == 0:
        summary,_=session.run([merged,optimizer],feed_dict=feed_dict)

        train_writer.add_summary(summary,step)
        print('Minibatch loss at step %d: %f' % (step,l))
        print('Minibatch accuracy: %.1f%%' % accuracy(predictions,batch_labels))
        print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(),valid_labels))
    except KeyboardInterrupt:
      last_weights=[p.eval() for p in params]
      np.savez('weights.npz',*last_weights)

      return last_weights
  
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(),test_labels))

  # 保存训练好的模型
  save_dir='models'
  if not os.path.exists(save_dir):
    os.makedirs(save_dir)
  save_path=os.path.join(save_dir,"deep_logo_model")
  saved=saver.save(session,save_path)
  print("Model saved in file: %s" % saved)